In [1]:
import pandas as pd
import numpy as np
import torchtext 
import torch
from sklearn.model_selection import train_test_split

In [2]:
data_train = pd.read_csv("clean_train.csv")
data_test = pd.read_csv("clean_test.csv")

In [3]:
%%time 
X_train = []
Y_train = []

curr_sent = 'Sentence: 1'
list_sent = []
list_y = []
for i in range(data_train.shape[0]):
    print("current {}/{}".format(i,data_train.shape[0]), end="\r")
    if data_train["Sentence #"][i] == curr_sent:
        list_sent.append(data_train['Word'][i])
        list_y.append(data_train['Tag'][i])
    else:
        curr_sent = data_train["Sentence #"][i]
        X_train.append(list_sent)
        Y_train.append(list_y)
        list_sent = [data_train['Word'][i]]
        list_y = [data_train['Tag'][i]]
    

CPU times: user 1min 28s, sys: 18.5 s, total: 1min 47s
Wall time: 1min 28s


In [4]:
%%time 
X_test = []
Y_test = []

curr_sent = 'Sentence: 35970'
list_sent = []
list_y = []
for i in range(data_test.shape[0]):
    print("current {}/{}".format(i,data_test.shape[0]), end="\r")
    if data_test["Sentence #"][i] == curr_sent:
        list_sent.append(data_test['Word'][i])
        list_y.append(data_test['Tag'][i])
    else:
        curr_sent = data_test["Sentence #"][i]
        X_test.append(list_sent)
        Y_test.append(list_y)
        list_sent = [data_test['Word'][i]]
        list_y = [data_test['Tag'][i]]

CPU times: user 37.4 s, sys: 7.62 s, total: 45 s
Wall time: 37.2 s


In [5]:
X_train = [[w for w in s] for s in X_train]
Y_train = [[t for t in s] for s in Y_train]

X_test = [[w for w in s] for s in X_test]
Y_test = [[t for t in s] for s in Y_test]

In [6]:
prova_train_x = [len(X_train[i]) for i in range(len(X_train))]
prova_train_y = [len(Y_train[i]) for i in range(len(Y_train))]
prova_train_x == prova_train_y

True

In [7]:
#remove strange words 

for i in range(len(X_train)): 
  
  for word in X_train[i]:
    
    if word == '\x85':
      idx = X_train[i].index(word) 
      X_train[i].remove(word)
      Y_train[i].pop(idx)
    
    if word == '\x94':
      idx = X_train[i].index(word) 
      X_train[i].remove(word)
      Y_train[i].pop(idx) 

In [8]:
X_train = [' '.join(X_train[i]) for i in range(len(X_train))]
X_test = [' '.join(X_test[i]) for i in range(len(X_test))]

Y_train = [' '.join(Y_train[i]) for i in range(len(Y_train))]
Y_test = [' '.join(Y_test[i]) for i in range(len(Y_test))]

In [9]:
train_df = pd.DataFrame(np.stack((X_train, Y_train), axis=-1), columns =['sequence', 'tag'])
train_df.head(10)

,sequence,tag
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Families of soldiers killed in the conflict jo...,O O O O O O O O O O O O O O O O O O B-per O O ...
2,They marched from the Houses of Parliament to ...,O O O O O O O O O O O B-geo I-geo O
3,"Police put the number of marchers at 10,000 wh...",O O O O O O O O O O O O O O O
4,The protest comes on the eve of the annual con...,O O O O O O O O O O O B-geo O O B-org I-org O ...
5,The party is divided over Britain 's participa...,O O O O O B-gpe O O O O B-geo O O O O O O O B-...
6,The London march came ahead of anti-war protes...,O B-geo O O O O O O O O O O O O B-geo O B-geo ...
7,The International Atomic Energy Agency is to h...,O B-org I-org I-org I-org O O O O O O O O B-ge...
8,Iran this week restarted parts of the conversi...,B-gpe O O O O O O O O O O B-geo O O O
9,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...


In [10]:
test_df = pd.DataFrame(np.stack((X_test, Y_test), axis=-1), columns =['sequence', 'tag'])
test_df.head(10)

,sequence,tag
0,Haitian police have killed a prominent rebel w...,B-gpe O O O O O O O O O O O O O O O O O B-tim ...
1,Officials said Sunday Jean Rene Anthony was ki...,O O B-tim B-per I-per I-per O O O O O O O O O ...
2,The gunbattle came a day after another rebel l...,O O O O O O O O O O B-per I-per O O O O O O O ...
3,"Mr. Ravix , the self-proclaimed leader of Hait...",B-per I-per O O O O O B-geo O O O O O O O O O ...
4,Haitian officials had been searching for the t...,B-gpe O O O O O O O O O O O O O O O O O O
5,Hurricane Ike roared through vital parts of th...,O B-per O O O O O O B-geo O O O O O O O O O O ...
6,Oil company officials say they are sending wor...,O O O O O O O O O O O O O O O O O O O
7,U.S. government officials say they are prepare...,B-geo O O O O O O O O O O O B-org I-org I-org ...
8,Concerns about possible supply shortages have ...,O O O O O O O O O O O O O O O B-geo O O O O O ...
9,"Meantime , three groups of insurance industry ...",O O O O O O O O O O O O O O O O O O O O O O O ...


In [11]:
train_df.to_csv('train_df_final.csv', index=False)
test_df.to_csv('test_df_final.csv', index=False)

In [12]:
data_train = pd.read_csv("train_df_final.csv")
prova_train_x2 = [data_train['sequence'][i].split() for i in range(len(data_train['sequence']))]
prova_train_y2 = [data_train['tag'][i].split() for i in range(len(data_train['tag']))]

prova_train_x3 = [len(prova_train_x2[i]) for i in range(len(prova_train_x2))]
prova_train_y3 = [len(prova_train_y2[i]) for i in range(len(prova_train_y2))]
#check length target and text is the same after removing strange words
prova_train_x3 == prova_train_y3

True

In [13]:
#define fields
sequence = torchtext.data.Field()
label = torchtext.data.Field(is_target=True)

fields = [('sequence', sequence), ('tag', label)] 

train_data, test_data = torchtext.data.TabularDataset.splits(path ='.', train= 'train_df_final.csv', test= 'test_df_final.csv', format = 'csv', fields = fields, skip_header = True)

In [14]:
#build vocabulary for training dataset using dictionary from torch text 
MAX_VOCAB_SIZE = 35000
sequence.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
label.build_vocab(train_data)

In [15]:
#length of vocabulary for training dataset (words + 1(unk) + 1(pad))
print(len(label.vocab))
print(len(sequence.vocab))

19
27311


In [16]:
print(label.vocab.stoi)
print(sequence.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f70cd3ffa60>, {'<unk>': 0, '<pad>': 1, 'O': 2, 'B-geo': 3, 'B-tim': 4, 'B-org': 5, 'I-per': 6, 'B-per': 7, 'I-org': 8, 'B-gpe': 9, 'I-geo': 10, 'I-tim': 11, 'B-art': 12, 'B-eve': 13, 'I-art': 14, 'I-eve': 15, 'I-gpe': 16, 'B-nat': 17, 'I-nat': 18})
defaultdict(<function _default_unk_index at 0x7f70cd3ffa60>, {'<unk>': 0, '<pad>': 1, 'the': 2, '.': 3, ',': 4, 'in': 5, 'of': 6, 'to': 7, 'a': 8, 'and': 9, 'The': 10, "'s": 11, 'for': 12, 'has': 13, 'is': 14, 'on': 15, 'that': 16, 'with': 17, 'have': 18, 'said': 19, 'was': 20, 'says': 21, 'by': 22, 'from': 23, 'at': 24, 'say': 25, 'as': 26, 'U.S.': 27, 'an': 28, 'are': 29, 'will': 30, '"': 31, 'were': 32, 'his': 33, 'Mr.': 34, 'government': 35, 'it': 36, 'been': 37, 'killed': 38, 'he': 39, 'people': 40, 'not': 41, 'after': 42, 'its': 43, 'officials': 44, 'President': 45, 'be': 46, 'two': 47, 'A': 48, 'also': 49, 'more': 50, 'United': 51, 'country': 52, 'military': 53, 'who': 54, 'they': 55, 'th

In [17]:
#the most common words of the vocabulary for test dataset 
sequence.vocab.freqs.most_common(10), label.vocab.freqs.most_common(10)

([('the', 30661),
  ('.', 27880),
  (',', 19347),
  ('in', 15384),
  ('of', 15342),
  ('to', 13589),
  ('a', 12065),
  ('and', 11708),
  ('The', 6618),
  ("'s", 6383)],
 [('O', 518907),
  ('B-geo', 21830),
  ('B-tim', 11790),
  ('B-org', 11600),
  ('I-per', 10261),
  ('B-per', 9943),
  ('I-org', 9560),
  ('B-gpe', 9323),
  ('I-geo', 4349),
  ('I-tim', 3770)])

In [18]:
#create batches for train and test data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 256

train_iterator = torchtext.data.BucketIterator(
    train_data,
    batch_size = BATCH_SIZE,
    shuffle=True,
    device=device)

test_iterator = torchtext.data.BucketIterator(
    test_data,
    shuffle=False,
    batch_size = BATCH_SIZE,
    device=device)

In [19]:
#check it works
batch = next(iter(train_iterator))
batch, batch.sequence, batch.tag

(
 [torchtext.data.batch.Batch of size 256]
 	[.sequence]:[torch.cuda.LongTensor of size 54x256 (GPU 0)]
 	[.tag]:[torch.cuda.LongTensor of size 54x256 (GPU 0)],
 tensor([[  322,    10,  3281,  ...,    62,    59,    97],
         [ 2839,  1177,  9634,  ...,    20,     8,    89],
         [   49,   555, 11962,  ...,   187,   159,    44],
         ...,
         [    1,     1,     1,  ...,     1,     1,     1],
         [    1,     1,     1,  ...,     1,     1,     1],
         [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:0'),
 tensor([[7, 2, 2,  ..., 2, 2, 9],
         [6, 2, 7,  ..., 2, 2, 2],
         [2, 2, 6,  ..., 2, 2, 2],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'))

In [20]:
#model
class RNN(torch.nn.Module):
    def __init__(self, vocab_dim, text, hidden_dim=256, output_dim=len(label.vocab), dropout=0.3, embedding_dim=256, bidirectional=True, num_layers=2):
        super().__init__()
        vocab = text.vocab
        self.embedding = torch.nn.Embedding(vocab_dim, embedding_dim)
        self.rnn = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, dropout=dropout, bidirectional=bidirectional)
        self.fc = torch.nn.Linear(hidden_dim*2 if bidirectional else hidden_dim, output_dim)
        
    def forward(self, sequence):

        #question = [sent len, batch size]        
        embedded = self.embedding(sequence)
    
        #embedded = [sent len, batch size, emb dim]        
        output, hidden = self.rnn(embedded)   
        
        #output = [sent len, batch size, hid dim]  
        L, B, H = output.shape
        output = self.fc(output.view(-1, H)) #B*L, H
        output = output.view(L, B, -1) #L, B, output_dim

        return output

In [21]:
#model to gpu
model = RNN(vocab_dim=len(sequence.vocab), text=sequence)
model.to(device)

RNN(
  (embedding): Embedding(27311, 256)
  (rnn): LSTM(256, 256, num_layers=2, dropout=0.3, bidirectional=True)
  (fc): Linear(in_features=512, out_features=19, bias=True)
)

In [22]:
#check dimensions of output
output = model(batch.sequence.to(device))
output.shape

torch.Size([54, 256, 19])

In [29]:
model = RNN(vocab_dim=len(sequence.vocab), text=sequence)
model.to(device)

epochs = 8
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, epochs+1):
    train_loss, train_acc = [], []
    model.train()
    for batch in train_iterator:
        optimizer.zero_grad()
        seq, tag = batch.sequence.to(device), batch.tag.to(device)
        output = model(seq) 
        #print(output.shape, tag.shape)
        L, B, C = output.shape
        #print(output.view(-1,C).shape, tag.view(-1).shape)
        loss = criterion(output.view(-1,C), tag.view(-1))
        loss.backward()
        optimizer.step()     
        train_loss.append(loss.item()) 
        acc = (torch.argmax(output, dim=2) == tag).sum().item() / (B*L)
        #print((torch.argmax(output, dim=1) == tag).sum())
        #print((tag.shape[0]*tag.shape[1]))
        train_acc.append(acc)
        
    test_loss, test_acc = [], []
    model.eval()
    with torch.no_grad():
        for batch in test_iterator:
            seq, tag = batch.sequence.to(device), batch.tag.to(device)
            output = model(seq) 
            L, B, C = output.shape
            loss = criterion(output.view(-1,C), tag.view(-1))  
            test_loss.append(loss.item()) 
            acc = (torch.argmax(output, dim=2) == tag).sum().item() / (B*L)
            test_acc.append(acc)
    print('Epoch: ' + str(epoch) + 
          ' Train loss: ' + str(np.mean(train_loss)) + 
          ' Test loss: ' + str(np.mean(test_loss)) +
          ' Train accuracy: ' + str(np.mean(train_acc)) +
          ' Test accuracy: ' + str(np.mean(test_acc)))

Epoch: 1 Train loss: 0.38777087500149554 Test loss: 0.19841686937403172 Train accuracy: 0.9138435500588973 Test accuracy: 0.9444887840473561
Epoch: 2 Train loss: 0.13019623505798253 Test loss: 0.08927408494847885 Train accuracy: 0.9644803648736164 Test accuracy: 0.9753165258247947
Epoch: 3 Train loss: 0.07309852272949435 Test loss: 0.06801621378102202 Train accuracy: 0.9792943927242517 Test accuracy: 0.9800795432262707
Epoch: 4 Train loss: 0.05323791808702729 Test loss: 0.06196571284152092 Train accuracy: 0.984338919024398 Test accuracy: 0.98193345151192
Epoch: 5 Train loss: 0.04201568849384785 Test loss: 0.058766682255775374 Train accuracy: 0.9874655791664425 Test accuracy: 0.9825813960419421
Epoch: 6 Train loss: 0.03430883042852987 Test loss: 0.06036057989013956 Train accuracy: 0.9896782818074433 Test accuracy: 0.9822877962469743
Epoch: 7 Train loss: 0.027732666852799327 Test loss: 0.06302887764065823 Train accuracy: 0.9914871052363912 Test accuracy: 0.9818802463218507
Epoch: 8 Train